**Accese google drive**





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## setup

In [ ]:
!pip install -qU transformers==4.48.3 datasets==3.2.0 optimum==1.24.0
!pip install -qU openai==1.61.0 wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import userdata
import wandb

wandb.login(key=userdata.get('wandb'))
hf_token = userdata.get('TOKEN_HUGINFACE')
!huggingface-cli login --token {hf_token}

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ahmed-ouka23456 (ahmed-ouka23456-ecole-nationale-de-l-intelligence-artifi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_http.py", line 409, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/whoami-v2

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/hf_api.py", line 1737, in whoami
    hf_raise_for_status(r)
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_http.py", line 482, in hf_raise_

## Knowledge Distillation


In [ ]:
raw_data_questions_path = "/content/drive/MyDrive/fine-tuning-PFA /data/totale_question.json"

with open(raw_data_questions_path, "r", encoding="utf-8") as f:
    raw_data_questions = json.load(f)

random.Random(100).shuffle(raw_data_questions)

print(f"Raw 1  data: {raw_data_questions[0]}")
print(f"Raw 1  data: {len(raw_data_questions)}")


Raw 1  data: {'question_fr': 'Qui peut accéder aux locaux pédagogiques en dehors des cours ? (variante 12)', 'question_ar': 'من يمكنه الوصول إلى المباني التعليمية خارج الدروس؟ (البديل 12)'}
Raw 1  data: 3099


In [ ]:
raw_data_questions[0]['question_fr']

'Qui peut accéder aux locaux pédagogiques en dehors des cours ? (variante 12)'

In [ ]:
raw_data_questions[0]['question_ar']

'من يمكنه الوصول إلى المباني التعليمية خارج الدروس؟ (البديل 12)'

In [ ]:
questions_fr = []
for question in raw_data_questions:
    questions_fr.append(question['question_fr'])

In [ ]:
questions_fr[0]


'Qui peut accéder aux locaux pédagogiques en dehors des cours ? (variante 12)'

## prompte pour la premier tach (fr-Question)

In [ ]:
cloud_model_id = "openai/gpt-4o-mini"
price_per_1m_input_tokens = 0.15
price_per_1m_output_tokens = 0.6

prompt_tokens = 0
completion_tokens = 0
save_to_fr = "/content/drive/MyDrive/fine-tuning-PFA /data/xsft.json"
save_to_ar = "/content/drive/MyDrive/fine-tuning-PFA /data/aRsft.json"

In [ ]:
!pip install json_repair

In [ ]:
import json_repair

def parse_json(text):
    try:
        return json_repair.loads(text)
    except:
        return None

In [ ]:
json_path = '/content/drive/MyDrive/fine-tuning-PFA /data/reponses_seulement.json'

with open(json_path, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

json_data_string = json.dumps(json_data, ensure_ascii=False, indent=2)



In [ ]:
json_data_string

'[\n  "École Nationale de l’Intelligence Artificielle et du Digital de Berkane.",\n  "Université Mohammed Premier.",\n  "Campus Mohamed Premier, Route de Fez, Berkane, Maroc.",\n  "+212 535 63 62 00.",\n  "contact@eniadb.ma.",\n  "https://eniadb.ma.",\n  "Facebook, LinkedIn, Instagram officiels (liens sur le site).",\n  "2019.",\n  "Partenariats avec l’industrie locale et centres de recherche universitaires.",\n  "Former des ingénieurs en IA et digital pour répondre aux besoins du marché numérique.",\n  "Faire de Berkane un pôle d’excellence en intelligence artificielle.",\n  "Innovation, éthique, inclusion, excellence académique.",\n  "Le Professeur X (nom à jour sur le site officiel).",\n  "Coordonnateurs de chaque filière listés dans le guide pédagogique.",\n  "Bâtiment B, 1er étage, bureau 12.",\n  "8h00–16h30 du lundi au vendredi.",\n  "Cycle préparatoire, cycle ingénieur en IA, Réseaux & Sécurité, Génie Informatique, Robotique & IoT.",\n  "Java, C++, Python, Bases de données, Sys

In [ ]:
json.dumps(parse_json(json_data_string), ensure_ascii=False),

('["École Nationale de l’Intelligence Artificielle et du Digital de Berkane.", "Université Mohammed Premier.", "Campus Mohamed Premier, Route de Fez, Berkane, Maroc.", "+212 535 63 62 00.", "contact@eniadb.ma.", "https://eniadb.ma.", "Facebook, LinkedIn, Instagram officiels (liens sur le site).", "2019.", "Partenariats avec l’industrie locale et centres de recherche universitaires.", "Former des ingénieurs en IA et digital pour répondre aux besoins du marché numérique.", "Faire de Berkane un pôle d’excellence en intelligence artificielle.", "Innovation, éthique, inclusion, excellence académique.", "Le Professeur X (nom à jour sur le site officiel).", "Coordonnateurs de chaque filière listés dans le guide pédagogique.", "Bâtiment B, 1er étage, bureau 12.", "8h00–16h30 du lundi au vendredi.", "Cycle préparatoire, cycle ingénieur en IA, Réseaux & Sécurité, Génie Informatique, Robotique & IoT.", "Java, C++, Python, Bases de données, Systèmes d’exploitation, Compétences culturelles, Françai

In [ ]:
ix = 0
for i in tqdm(range(400, len(questions_fr))):

    prompte_Task_1_messages = [
     {
        "role": "system",
        "content": "\n".join([
            "Vous êtes un assistant dans une école qui s'appelle l'École Nationale de l'Intelligence Artificielle et Digitale de Berkane, capable de répondre aux questions des étudiants.",
            "Vous recevrez des informations sur l'école sous forme de JSON ainsi que des textes contenant des exemples de questions courantes et de leurs réponses.",
            "Certaines réponses aux questions proviennent exclusivement de la base de données Q&R via la fonction get_school_info. Cependant, ce sont des réponses directes. Assurez-vous d'expliquer."
            "Vérifiez les données attentivement lorsque vous répondez.",
            "Essayez d'éviter tous les mots et textes qui n'ont pas de sens dans ces données.",
            "Faites très attention à la langue dans laquelle la question est posée.",
            "Vous devez répondre dans la même langue que celle de la question.",
            "Ne répondez pas tant que vous n’êtes pas sûr de la langue de la question.",
            "Si c'est en arabe, répondez en arabe. Si c'est en français, répondez en français. Si c'est en anglais, répondez en anglais.",
            "Ignorez les éléments inutiles dans la question tels que les numéros de version ou de commande, et concentrez-vous uniquement sur la question.",
            "Faites attention aux fautes d'orthographe pour ne pas altérer votre compréhension.",
            "Extraire les détails JSON du texte conformément aux questions posées et aux spécifications Pydantic.",
            "Extraire les détails comme indiqué dans le texte. Vous pouvez les reformater, mais gardez le sens.",
            "Ne pas générer d'introduction ni de conclusion.",
            "",
            "## Texte PDF Principal:",
            pdf_text.strip(),
            "",
            "## Texte CNPQ:",
            pdf_text_cnpn.strip(),
            "",
            "## donnés :",
            json.dumps(parse_json(json_data_string), ensure_ascii=False),
            "",


        ])
    },
    {
        "role": "user",
        "content": "\n".join([
            "## Question:",
            questions_fr[i].strip(),
            "",

            "## Pydantic Details:",
            json.dumps(
                ChatBotResponse.model_json_schema(), ensure_ascii=False
            ),
            "",

            "## Repense Details:",
            "```json"
        ])
    }
]

    completion = openai_client.chat.completions.create(
        messages=prompte_Task_1_messages,
        model="openai/gpt-4o-mini",
        temperature=0.2,

    )

    if not completion or not completion.choices or not completion.choices[0].message or not completion.choices[0].message.content:
            print(f"Aucune réponse valide pour la question {ix}")

    llm_response = completion.choices[0].message.content
    llm_resp_dict = parse_json(llm_response)

    # if not llm_resp_dict:
    #         continue
    with open(save_to_fr, "a", encoding="utf8") as dest:
            dest.write(json.dumps({
                "id": ix,
                "question": questions_fr[i].strip(),
                "task": "extraire des informations sur l'école",
                "output_scheme": json.dumps( ChatBotResponse.model_json_schema(), ensure_ascii=False ),
                "response": llm_resp_dict,
            }, ensure_ascii=False, default=str)  + "\n" )

    ix += 1
    prompt_tokens += completion.usage.prompt_tokens
    completion_tokens += completion.usage.completion_tokens

    if(ix % 3) == 0:
            cost_input = (prompt_tokens / 1_000_000) * price_per_1m_input_tokens
            cost_output = (completion_tokens / 1_000_000) * price_per_1m_output_tokens
            total_cost = cost_input + cost_output

            print(f"Iteration {ix}: Total Cost = ${total_cost:.4f} ")


  0%|          | 0/8897 [00:00<?, ?it/s]

Iteration 3: Total Cost = $7.9594 
Iteration 6: Total Cost = $7.9691 
Iteration 9: Total Cost = $7.9788 
Iteration 12: Total Cost = $7.9885 
Iteration 15: Total Cost = $7.9981 
Iteration 18: Total Cost = $8.0078 
Iteration 21: Total Cost = $8.0175 
Iteration 24: Total Cost = $8.0272 
Iteration 27: Total Cost = $8.0369 
Iteration 30: Total Cost = $8.0466 
Iteration 33: Total Cost = $8.0563 
Iteration 36: Total Cost = $8.0660 
Iteration 39: Total Cost = $8.0758 
Iteration 42: Total Cost = $8.0855 
Iteration 45: Total Cost = $8.0952 
Iteration 48: Total Cost = $8.1050 
Iteration 51: Total Cost = $8.1147 
Iteration 54: Total Cost = $8.1244 
Iteration 57: Total Cost = $8.1341 
Iteration 60: Total Cost = $8.1438 
Iteration 63: Total Cost = $8.1535 
Iteration 66: Total Cost = $8.1632 
Iteration 69: Total Cost = $8.1729 
Iteration 72: Total Cost = $8.1826 
Iteration 75: Total Cost = $8.1923 
Iteration 78: Total Cost = $8.2020 
Iteration 81: Total Cost = $8.2117 
Iteration 84: Total Cost = $8.2

KeyboardInterrupt: 

In [ ]:
questions_ar = []
for question in raw_data_questions:
    questions_ar.append(question['question_ar'])

In [ ]:
ix = 0
for i in tqdm(range(1200, len(questions_fr))):


    prompte_Task_2_messages = [
     {
        "role": "system",
        "content": "\n".join([
            "Vous êtes un assistant dans une école qui s'appelle l'École Nationale de l'Intelligence Artificielle et Digitale de Berkane, capable de répondre aux questions des étudiants.",
            "Vous recevrez des informations sur l'école sous forme de JSON ainsi que des textes contenant des exemples de questions courantes et de leurs réponses.",
            "Certaines réponses aux questions proviennent exclusivement de la base de données Q&R via la fonction get_school_info. Cependant, ce sont des réponses directes. Assurez-vous d'expliquer."
            "Vérifiez les données attentivement lorsque vous répondez.",
            "Essayez d'éviter tous les mots et textes qui n'ont pas de sens dans ces données.",
            "Faites très attention à la langue dans laquelle la question est posée.",
            "Vous devez répondre dans la même langue que celle de la question.",
            "Ne répondez pas tant que vous n’êtes pas sûr de la langue de la question.",
            "Si c'est en arabe, répondez en arabe. Si c'est en français, répondez en français. Si c'est en anglais, répondez en anglais.",
            "Ignorez les éléments inutiles dans la question tels que les numéros de version ou de commande, et concentrez-vous uniquement sur la question.",
            "Faites attention aux fautes d'orthographe pour ne pas altérer votre compréhension.",
            "Extraire les détails JSON du texte conformément aux questions posées et aux spécifications Pydantic.",
            "Extraire les détails comme indiqué dans le texte. Vous pouvez les reformater, mais gardez le sens.",
            "Ne pas générer d'introduction ni de conclusion.",
            "",
            "## Texte PDF Principal:",
            pdf_text.strip(),
            "",
            "## Texte CNPQ:",
            pdf_text_cnpn.strip(),
            "",
            "## donnés :",
            json.dumps(parse_json(json_data_string), ensure_ascii=False),
            "",


        ])
    },
    {
        "role": "user",
        "content": "\n".join([
            "## Question:",
            questions_ar[i].strip(),
            "",

            "## Pydantic Details:",
            json.dumps(
                ChatBotResponse.model_json_schema(), ensure_ascii=False
            ),
            "",

            "## Repense Details:",
            "```json"
        ])
    }
]

    completion = openai_client.chat.completions.create(
        messages=prompte_Task_2_messages,
        model="meta-llama/llama-4-scout",
        temperature=0.2,

    )

    if not completion or not completion.choices or not completion.choices[0].message or not completion.choices[0].message.content:
            print(f"Aucune réponse valide pour la question {ix}")

    llm_response = completion.choices[0].message.content
    llm_resp_dict = parse_json(llm_response)

    # if not llm_resp_dict:
    #         continue
    with open(save_to_ar, "a", encoding="utf8") as dest:
            dest.write(json.dumps({
                "id": ix,
                "question": questions_ar[i].strip(),
                "task": "extraire des informations sur l'école",
                "output_scheme": json.dumps( ChatBotResponse.model_json_schema(), ensure_ascii=False ),
                "response": llm_resp_dict,
            }, ensure_ascii=False, default=str)  + "\n" )

    ix += 1
    prompt_tokens += completion.usage.prompt_tokens
    completion_tokens += completion.usage.completion_tokens

    if(ix % 3) == 0:
            cost_input = (prompt_tokens / 1_000_000) * price_per_1m_input_tokens
            cost_output = (completion_tokens / 1_000_000) * price_per_1m_output_tokens
            total_cost = cost_input + cost_output

            print(f"Iteration {ix}: Total Cost = ${total_cost:.4f} ")

  0%|          | 0/8097 [00:00<?, ?it/s]

Iteration 3: Total Cost = $13.7631 
Iteration 6: Total Cost = $13.7733 
Iteration 9: Total Cost = $13.7832 
Iteration 12: Total Cost = $13.7933 
Iteration 15: Total Cost = $13.8034 
Iteration 18: Total Cost = $13.8134 
Iteration 21: Total Cost = $13.8233 
Iteration 24: Total Cost = $13.8330 
Iteration 27: Total Cost = $13.8430 
Iteration 30: Total Cost = $13.8531 
Iteration 33: Total Cost = $13.8629 
Iteration 36: Total Cost = $13.8733 
Iteration 39: Total Cost = $13.8830 
Iteration 42: Total Cost = $13.8928 
Iteration 45: Total Cost = $13.9027 
Iteration 48: Total Cost = $13.9125 
Iteration 51: Total Cost = $13.9225 
Iteration 54: Total Cost = $13.9325 
Iteration 57: Total Cost = $13.9423 
Iteration 60: Total Cost = $13.9525 
Iteration 63: Total Cost = $13.9625 
Iteration 66: Total Cost = $13.9724 
Iteration 69: Total Cost = $13.9823 
Iteration 72: Total Cost = $13.9922 
Iteration 75: Total Cost = $14.0020 
Iteration 78: Total Cost = $14.0118 
Iteration 81: Total Cost = $14.0216 
Iter